### LSTM  model


### Import packages

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tokenization
import tensorflow as tf
import tensorflow_hub as hub
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix


2022-12-01 13:10:26.648257: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


from keras.models import Sequential
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd

In [3]:
Twitter_stock=pd.read_csv("../Data/combind_sen_stock.csv")

Twitter_stock.head()

FileNotFoundError: [Errno 2] No such file or directory: '../Data/combind_sen_stock.csv'

In [10]:
df=pd.DataFrame([Twitter_stock["Tweets_clean_more"],Twitter_stock["Label"]]).T


In [29]:
label_encoder = LabelEncoder()

df["Label"] = label_encoder.fit_transform(df["Label"])

df

,Tweets_clean_more,Label
0,i made volum model s go now i need work miniat...,1
1,yeah realli mom attest not sure i handl char m...,0
2,model s option perform red black i deliv car j...,1
3,if ever want know real truth moon land best on...,0
4,cool person essay it realli reson i felt way c...,1
...,...,...
1417,it remov simul improv render resolut congrat ...,0
1418,tesla build collis repair capabl help address ...,1
1419,th raptor engin come soon mayb use get that ve...,0
1420,one mani reason need make life multiplanetari ...,0


### Get the training and testing dataset

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df["Tweets_clean_more"], df["Label"], test_size=0.2)

print('X_train shape: ', X_train.shape)
print('X_val shape: ', X_val.shape)
print('y_train shape: ', y_train.shape)
print('y_val shape: ', y_val.shape)

X_train shape:  (1137,)
X_val shape:  (285,)
y_train shape:  (1137,)
y_val shape:  (285,)


In [31]:
print('Train Class Proportion:\n', y_train.value_counts() / len(y_train) * 100)
print('\nValidation Class Proportion:\n', y_val.value_counts() / len(y_val) * 100)

Train Class Proportion:
 0    49.076517
1    48.372911
2     2.550572
Name: Label, dtype: float64

Validation Class Proportion:
 1    48.421053
0    47.368421
2     4.210526
Name: Label, dtype: float64


In [32]:


tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df["Tweets_clean_more"])
xtrain= tokenizer.texts_to_sequences(X_train)
xtest= tokenizer.texts_to_sequences(X_val) 




In [48]:
vocab_size=len(tokenizer.word_index)+1


 
maxlen = 50

X_train_text = pad_sequences(xtrain, padding='post', maxlen=maxlen)
X_val_text = pad_sequences(xtest, padding='post', maxlen=maxlen)

print('X_train shape:', X_train_text.shape)
print('X_train shape:', X_val_text.shape)



X_train shape: (1137, 50)
X_train shape: (285, 50)


In [62]:
embedding_dim=50
model=Sequential()
model.add(layers.Embedding(input_dim=vocab_size,
         output_dim=embedding_dim,
         input_length=maxlen))
model.add(layers.LSTM(units=50,return_sequences=True))
model.add(layers.LSTM(units=10))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10))
model.add(layers.Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy", 
     metrics=['accuracy'])
model.summary()
model.fit(X_train_text,y_train, epochs=20, batch_size=16, verbose=False)



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 50, 50)            368000    
                                                                 
 lstm_8 (LSTM)               (None, 50, 50)            20200     
                                                                 
 lstm_9 (LSTM)               (None, 10)                2440      
                                                                 
 dropout_4 (Dropout)         (None, 10)                0         
                                                                 
 dense_8 (Dense)             (None, 10)                110       
                                                                 
 dense_9 (Dense)             (None, 1)                 11        
                                                                 
Total params: 390,761
Trainable params: 390,761
Non-tr

In [66]:
loss, acc = model.evaluate(X_train_text, y_train, verbose=False)
print("Training Accuracy: ", acc)
loss, acc = model.evaluate(X_val_text, y_val, verbose=False)
print("Test Accuracy: ", acc)





Training Accuracy:  0.48372912406921387
Test Accuracy:  0.4842105209827423


In [52]:
ypred=model.predict(X_val_text)


ypred[ypred>0.5]=1 
ypred[ypred<=0.5]=0 

9/9 [==============================] - 0s 8ms/step


In [54]:
cm = confusion_matrix(y_val, ypred)
cm

array([[  0, 135,   0],
       [  0, 138,   0],
       [  0,  12,   0]])

In [59]:
print(classification_report(y_val,ypred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       135
           1       0.48      1.00      0.65       138
           2       0.00      0.00      0.00        12

    accuracy                           0.48       285
   macro avg       0.16      0.33      0.22       285
weighted avg       0.23      0.48      0.32       285



/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Results:
The LSTM model show us that the predict result. 



In [ ]:
pd.read_csv()